### 1) Shadow State Tomography

In [ ]:
def get_basis_unitary(basis):
    
    if basis == "X":
        unitary = 1/np.sqrt(2) * np.array([[1.,1.],
                                           [1.,-1.]])
    elif basis == "Y":
        unitary = 1/np.sqrt(2) * np.array([[1.,-1.0j],
                                           [1.,1.j]])
    elif basis == "Z":
        unitary = np.eye(2)
        
    return unitary

In [ ]:
def get_inverted_channel(channel, qubits_count):
    
    identity = np.eye(2 ** qubits_count)
    
    inverted_channel = (2 ** qubits_count + 1) * channel - identity
    
    return inverted_channel

In [ ]:
def build_bases_rotation_circuit(bases):
    
    qubits_count = len(bases)
    
    circuit = qiskit.QuantumCircuit(qubits_count)
    
    for qubit, basis in enumerate(bases):
        
        if basis == "X":
            circuit.h(qubit)
        
        elif basis == "Y":
            circuit.sdg(qubit)
            circuit.h(qubit)
        
        elif basis == "Z":
            pass
        
    return circuit

In [ ]:
def run_shadow_circuits(shadow_circuits, backend, shots_counts):

    counts_list = []

    for shadow_circuit, shots in zip(shadow_circuits, shots_counts):

        job = backend.run(shadow_circuit, shots=shots)

        counts = job.result().get_counts()

        counts_list.append(counts)
        
    return counts_list

In [ ]:
def get_shadows(bases_list, counts_list):

    shadows = []

    for bases, counts in zip(bases_list, counts_list):

        for state, count in counts.items():

            shadow_part = 1

            for bit_index, bit in enumerate(state[::-1]):

                basis = bases[bit_index]

                basis_unitary = get_basis_unitary(basis)

                vector = basis_unitary[int(bit), :]

                outer_product = np.outer(vector.conj(), vector)

                inverted_channel = get_inverted_channel(outer_product, 1)

                shadow_part = np.kron(inverted_channel, shadow_part)

            shadow = shadow_part * count

            shadows.append(shadow)
            
    return shadows

In [ ]:
def plot_density_matrix(density_matrix, label=""):

    plt.suptitle(f"{label} Density Matrix", y=0.9)

    real = plt.subplot(121)
    real.set_title("Real part")

    real.imshow(density_matrix.real, vmin=-0.7, vmax=0.7)

    real.axes.set_xticks(())
    real.axes.set_yticks(())


    imaginary = plt.subplot(122)
    imaginary.set_title("Imaginary part")
    imaginary.imshow(density_matrix.imag, vmin=-0.7, vmax=0.7)

    imaginary.axes.set_xticks(())
    imaginary.axes.set_yticks(())

    plt.show()

In [ ]:
def get_mean_square_error(a, b):

    mean_square_error = np.abs(((a - b) ** 2).mean())
    
    return mean_square_error

### 2) Quantum Fourier Transform

In [105]:
def build_state_circuit(state):
    
    qubits_count = len(state)
    
    state_circuit = qiskit.QuantumCircuit(qubits_count)
    
    for index, bit in enumerate(state):
        
        if bit == "1":
            position = qubits_count - index - 1
            state_circuit.x(position)

    state_circuit.barrier(label="State")
    
    return state_circuit

In [106]:
def build_qft_circuit(qubits_count):
    
    qubits = list(range(qubits_count))
    
    circuit = qiskit.QuantumCircuit(qubits_count)
    
    for control_qubit in reversed(qubits):
        
        for target_qubit in reversed(range(control_qubit, qubits_count)):
            
            if control_qubit == target_qubit:
                
                circuit.h(control_qubit)
                
            else:
                
                theta = np.pi / 2 ** (target_qubit - control_qubit)
                
                circuit.crz(theta, control_qubit, target_qubit)
                
        circuit.barrier()
        
    return circuit

In [107]:
def build_fourrier_adder(qubits_count, number_to_add=1):
    
    circuit = qiskit.QuantumCircuit(qubits_count)
    
    for qubit in range(qubits_count):
        
        angle = number_to_add * (np.pi / 2 ** qubit)
        
        circuit.rz(angle, qubit)
        
    circuit.barrier(label="Adder")
    
    return circuit

In [ ]:
def build_counts_figure(initial_states, measured_states, counts,
                        elevation=10, initial_azimuth=230):

    initial_states_grid, measured_states_grid = np.meshgrid(initial_states, measured_states)

    figure, ax = plt.subplots(figsize=(5, 5),
                              subplot_kw={"projection": "3d"})

    surface = ax.plot_surface(initial_states_grid,
                              measured_states_grid,
                              counts.T,
                              cmap='plasma',
                              linewidth=30, 
                              antialiased=True)

    ax.view_init(elev=elevation, azim=initial_azimuth)

    ax.zaxis.set_rotate_label(False)
    
    ax.set_xlabel('Initial State Index')
    ax.set_ylabel('Measured State Index')
    ax.set_zlabel('Count', rotation=90)
        
    ax.patch.set_alpha(0)
        
    figure.patch.set_alpha(0)
    
    # figure.patch.set_facecolor('black')
    
    plt.close()
    
    return figure